# Tutorial

We can illustrate how to use hourly by looking at the hourly repo itself.

    git clone https://github.com/asherp/hourly.git
    cd hourly

In [24]:
from hourly.hourly import get_work_commits, get_labor, get_earnings

In [25]:
from hourly import hourly

```get_work_commits``` gathers all commits into a pandas array

In [26]:
work = hourly.get_work_commits('..')
work.head()

,message,hash
time,,
2018-10-19 23:40:41-04:00,Initial commit,ef5690543bfb354b9325d1fbd1f9abbafbb4d9a4
2018-10-19 23:57:48-04:00,clock in,5c8f05b57b739ec525291c248ea9200651b49997
2018-10-20 00:21:40-04:00,preparing setup.py,254ecdacb52fc70bc358f8d55be58df3b70c7609
2018-10-20 00:39:11-04:00,clock out - work done for the day,0e33fa3d74f663f954b05dd9f30e0128ca7af162
2018-10-20 01:06:08-04:00,clock in - start adding requirements and examp...,dc065b17337b14c2f8e0458de61e6880a338d6ae


```get_labor``` calculates hours worked by differencing commit timestamps and raises an error if clock in and clock out are of different lengths.

### Getting time card

In [29]:
labor = hourly.get_labor(work, end_date='2018-10-20 02:11:54-04:00')
labor

pay period: 2018-10-19 23:57:48-04:00 -> 2018-10-20 02:11:54-04:00


,TimeIn,log in,TimeOut,log out,TimeDelta
0,2018-10-19 23:57:48-04:00,clock in,2018-10-20 00:39:11-04:00,clock out - work done for the day,00:41:23
1,2018-10-20 01:06:08-04:00,clock in - start adding requirements and examp...,2018-10-20 01:47:01-04:00,clock out,00:40:53
2,2018-10-20 01:47:45-04:00,clock in - pro bono,2018-10-20 01:51:36-04:00,clock out - pro bono,00:03:51
3,2018-10-20 02:03:56-04:00,clock in - finishing tutorial,2018-10-20 02:11:54-04:00,clock out - converted notebook for README,00:07:58


In [41]:
tstart, tend = labor.iloc[[0,-1]].TimeIn
tstart

Timestamp('2018-10-19 23:57:48-0400', tz='US/Eastern')

### Handling errant clock in/out messages
If you mistakenly put "clock out" in a message, hourly will interpret the message as a legitimate end time. This will likely raise an error when computing the labor. For example, there is a problematic commit in the this repo's history:

In [6]:
problematic_commit = work[work.hash == 'd9ec537b36475b565df6b28d0cab6edc3a89f2da']
problematic_commit

,message,hash
time,,
2018-10-20 02:14:21-04:00,had to clock out so notebook examples don't break,d9ec537b36475b565df6b28d0cab6edc3a89f2da


When we include this in our labor calculation, we get the following error:

In [16]:
try:
    hourly.get_labor(work, end_date = '2018-10-20 13:16:13-04:00')
except ValueError as e:
    print(e)

pay period: 2018-10-19 23:57:48-04:00 -> 2018-10-20 13:16:13-04:00
In/Out logs do not match


We can skip this errant commit by setting ```errant_clocks```

In [17]:
hourly.get_labor(work, end_date = '2018-10-20 13:16:13-04:00', 
          errant_clocks = ['d9ec537b36475b565df6b28d0cab6edc3a89f2da'])

pay period: 2018-10-19 23:57:48-04:00 -> 2018-10-20 13:16:13-04:00


,TimeIn,log in,TimeOut,log out,TimeDelta
0,2018-10-19 23:57:48-04:00,clock in,2018-10-20 00:39:11-04:00,clock out - work done for the day,00:41:23
1,2018-10-20 01:06:08-04:00,clock in - start adding requirements and examp...,2018-10-20 01:47:01-04:00,clock out,00:40:53
2,2018-10-20 01:47:45-04:00,clock in - pro bono,2018-10-20 01:51:36-04:00,clock out - pro bono,00:03:51
3,2018-10-20 02:03:56-04:00,clock in - finishing tutorial,2018-10-20 02:11:54-04:00,clock out - converted notebook for README,00:07:58
4,2018-10-20 11:53:00-04:00,clock in - handling errant messages,2018-10-20 13:16:13-04:00,clock out - converting to pd.Timestamp,01:23:13


### Filtering work session keywords

Use the "ignore" key word to skip any work you don't want to include in your invoices.

In [44]:
labor = hourly.get_labor(work, 
            ignore = 'pro bono', 
            end_date = '2018-10-20 13:16:13-04:00',
            errant_clocks = ['d9ec537b36475b565df6b28d0cab6edc3a89f2da'])
labor

pay period: 2018-10-19 23:57:48-04:00 -> 2018-10-20 13:16:13-04:00
ignoring pro bono


,TimeIn,log in,TimeOut,log out,TimeDelta
0,2018-10-19 23:57:48-04:00,clock in,2018-10-20 00:39:11-04:00,clock out - work done for the day,00:41:23
1,2018-10-20 01:06:08-04:00,clock in - start adding requirements and examp...,2018-10-20 01:47:01-04:00,clock out,00:40:53
3,2018-10-20 02:03:56-04:00,clock in - finishing tutorial,2018-10-20 02:11:54-04:00,clock out - converted notebook for README,00:07:58
4,2018-10-20 11:53:00-04:00,clock in - handling errant messages,2018-10-20 13:16:13-04:00,clock out - converting to pd.Timestamp,01:23:13


## Get total earnings

Total earnings can be found using this function. Currency is just a string for printing, but in the future we can add unit conversion.

In [19]:
hourly.get_earnings(labor, wage = 30, currency = 'USD')

0 days 02:53:27, 2.89 hours worked
86.72 USD


86.72

# Time adjustment

If you forget to clock in or clock out, you can correct your time sheet by adjusting your clock time. By default, Hourly adjusts time stamps automatically when a commit message contains the keyword "T-". Set ```correct_times``` to false to override this behavior.

In [21]:
work = hourly.get_work_commits('..', correct_times=False) # reports the actual time of the commit
work = work[work.message.str.contains('T-')] #filter by T-
work

,message,hash
time,,
2019-02-25 11:19:10-05:00,clock in T-1hr,d7add63b4d2e3e1ca1423296aaed25d9c28944da
2019-02-25 12:54:51-05:00,clock out T-5m,acfb8596317786e38177345aa2531098043a5c49


In [22]:
work = get_work_commits('..') # reports actual work time, according to commit message
work = work[work.message.str.contains('T-')] #filter by T-
work

,message,hash
time,,
2019-02-25 10:19:10-05:00,clock in T-1hr,d7add63b4d2e3e1ca1423296aaed25d9c28944da
2019-02-25 12:49:51-05:00,clock out T-5m,acfb8596317786e38177345aa2531098043a5c49


!!! Note
    Only time subtractions are supported for now.

!!! Tip Time delta formats
    Hourly uses pandas' Timedelta format to modify times. This allows for syntax like ```clock in T-1hr45```